# Encontrando números primos

Encontrar numeros primos siempre ha representado un reto, pues aunque su definición parezca trivial, entre más grande sea un número $n$ mayor es el número de interaciones necesarias para poder comprobar si este número es o no un número primo. Primero recordamos que un número primo es un numero $n\in \mathbb{N} $ tál que $ \forall\  a \in \mathbb{N},\ a\in (1,n)$, $n/a \not\in \mathbb{N}$

Una forma trivial de atacar este problema es por inspección, para esto hacemos una función que pruebe pueda comprobar si un numero dado $n$ es primo o no.

In [1]:
import numpy as np
import math
import time
from collections import Counter

In [2]:
def basic_is_prime(n):
    result=True
    for i in range(2,n):
        if n%i==0:
            result = False
    return result

Esta función recibe un numero entero $n$ y revisa todos los números hasta $n-1$ buscando los posibles divisores de $n$ en el caso en el que ningún número divida a $n$ entre $2$ y $n-1$, se concluye entonces que $n$ es primo y por ende la función `basic_is_prime`retorna `True`.
Ahora vamos a usar esta función para encontrar los primeros números primos entre $0$ y $50000$.

In [3]:
start=time.time()
lista_basic_prime=[]
for i in range(2,50000):
    if basic_is_prime(i):
        lista_basic_prime.append(i)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 91.24638080596924


Como se ve, esto es un algoritmo lento, si se quisiera determinar si un número grande es o no un número primo esto podría tardar demasiado, con lo cual nos lleva a considerar un algoritmo distinto para poder determinar si algún numero es primo o no.

## Algoritmo mejorado

suponga que $m=\sqrt{n}$, entonces $m\times m = n$. Ahora si $n$ no es un número primo entonces $n$ puede ser escrito como $a\times b \Rightarrow \ m\times m=a\times b$. Observemos que si $m$ es un numero real, mientras que n,a y b son numeros naturales.

Ahora tenemos 3 posibles casos:
1. $a>m \Rightarrow b<m$
2. $a=m \Rightarrow b=m$
3. $a<m \Rightarrow b>m$

Para todos estos 3 casos, se tiene que $\min (a,b)\leq m$. Por ende $m$ será una cota para encontrar al menos un factor de $n$, lo cual resulta ser una condición suficiente para mostrar que $n$ no es primo.

In [4]:
def new_is_prime(n):
    result=True
    for i in range(2,int(np.sqrt(n)+1)):
        if n%i==0:
            result = False
    return result

In [5]:
start=time.time()
lista_new_prime=[]
for i in range(2,50000):
    if new_is_prime(i):
        lista_new_prime.append(i)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 0.697455883026123


Como se puede ver esto es un mejora significativa en el programa, sin embargo esto podría ser mejorado ligeramente más. Para esto se hace uso del hecho de que ningun número par puede ser un número primo, con lo cual la busqueda de puede hacerce de en los impares.

In [6]:
def list_of_primes(n):
    lista=np.array([2])
    for i in range(3,n+1,2):
        if new_is_prime(i):
            lista=np.append(lista,i)
    return lista

In [7]:
start=time.time()
lista_new_prime_2=list_of_primes(50000)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 0.34560537338256836


In [8]:
len(lista_basic_prime),len(lista_new_prime),len(lista_new_prime_2)

(5133, 5133, 5133)

Una de las aplicaciones que tiene el encontrar los numeros primos es la descomposición de un número en factores primos, una forma de implementar esto se podría hacer de la siguiente forma: 

------ Sugerir como ejercicio ------


In [9]:
lista= list_of_primes(100000)

In [14]:
def get_prime_Factors(n):
    if type(n) is not int or n < 1:
        # Not an integer, negative or 0
        return []
    elif(n<3):
        return [n]
    factors=[]
    if new_is_prime(n):
        return [n]
    while(n>1):
        for i in range(2,n+1):
            if new_is_prime(i) and not n % i:
                n=int(n/i)
                factors.append(i)
                break
    return factors

In [28]:
def primeFactors(n):
    f= get_prime_Factors(n)
    factors = sorted(list(set(f)))
    powers = [f.count(factor) for factor in factors]
    result=""
    for i in range(len(factors)):
        if powers[i]==1:
            result+="("+str(factors[i])+")"
        else:
            result+="("+str(factors[i])+"**"+str(powers[i])+")"
    return result

In [29]:
primeFactors(5031)

'(3**2)(13)(43)'

In [ ]:
def primeFactors(n):
    resto=n
    continuar=True
    val=[]
    while(continuar):
        for i in lista:
            if resto%i==0:
                resto=resto/i
                val.append(i)
                break        
        if resto==1:
            continuar=False
    dicc=Counter(val)
    result=""
    for i in sorted(dicc):
        if dicc[i]==1:
            result+="("+str(i)+")"
        else:
            result+="("+str(i)+"**"+str(dicc[i])+")"
    return result

Usemos la anterior función para calcular la descomposición en numeros primos de $512345021, 777546031$ y $7775460$

In [13]:
print(primeFactors(512345021),primeFactors(777546031),primeFactors(7775460))

(512345021) (23567)(32993) (2**2)(3**3)(5)(7)(11**2)(17)


In [15]:
def get_unique_prime_factors_with_count(n):
    """Get a list containing two lists: one with prime numbers appearing in the
    decomposition and the other containing their respective power.
    Keyword arguments:
    n -- integer value to get the lists for
    Return a list of two lists, e.g.: [[2,5],[2,2]].
    """
    f = get_all_prime_factors(n)
    factors = list(set(f))
    powers = [f.count(factor) for factor in factors]
    return [factors, powers]

In [15]:
def get_unique_prime_factors_with_products(n):
    """Get a list of prime factors raised to the power of their respective
    powers.
    Keyword arguments:
    n -- integer value to get the list for
    Return a list, e.g.: [4,25].
    """
    cf = get_unique_prime_factors_with_count(n)
    return [factor**count for factor, count in zip(cf[0], cf[1])]

In [15]:
def is_prime(n):
    """Check if given number is a prime number.
    Keyword arguments:
    n -- integer value to check if it is a prime number
    Return True is the number is prime, otherwise False.
    """
    for i in range(2, n):
        if not n % i:
            return False
    return True

'(8051.2)'

In [25]:
if not 3%3:
    print("hola")

hola


# Algoritmo de Pollard's Rho

In [34]:
def g(x,n):
    return (x**2 +1)%n

In [ ]:
n=777546031
x=np.random.randint(2,n)
y=np.random.randint(2,n)
d=1
while(d==1):
    #print(x,y)
    x= g(x,n)
    y=g(g(y,n),n)
    d=math.gcd(np.abs(x-y),n)
print(d)

6984

[link para hacer el algoritmo de exponenciación](https://eli.thegreenplace.net/2009/03/28/efficient-modular-exponentiation-algorithms)